In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import psutil


class VAE(nn.Module):
    def __init__(self, latent_dim=200):
        super(VAE, self).__init__()

        # Encoder: 4 Conv2d layers with stride=2, kernel=4, padding=1
        # Input: (3, 64, 64) -> (32, 32, 32) -> (64, 16, 16) -> (128, 8, 8) -> (256, 4, 4)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),    # 0
            nn.ReLU(),                                                # 1
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),   # 2
            nn.ReLU(),                                                # 3
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 4
            nn.ReLU(),                                                # 5
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # 6
            nn.ReLU(),                                                # 7
        )

        # 256 * 4 * 4 = 4096
        self.fc_mu = nn.Linear(4096, latent_dim)
        self.fc_logvar = nn.Linear(4096, latent_dim)

        # Decoder input
        self.decoder_input = nn.Linear(latent_dim, 4096)

        # Decoder: 4 ConvTranspose2d layers
        # (256, 4, 4) -> (128, 8, 8) -> (64, 16, 16) -> (32, 32, 32) -> (3, 64, 64)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 0
            nn.ReLU(),                                                          # 1
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # 2
            nn.ReLU(),                                                          # 3
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),    # 4
            nn.ReLU(),                                                          # 5
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),     # 6
            nn.Sigmoid(),                                                       # 7
        )

    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten to (batch, 4096)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        x = self.decoder_input(z)
        x = x.view(x.size(0), 256, 4, 4)  # Reshape to (batch, 256, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy
from torch.optim import AdamW
from torch.distributions import Laplace
import numpy as np
import random

# --- Classe Principale Laplacian DDPM ---

class LaplacianVAE:
    def __init__(self, model, device="cuda"):
        self.device = device
        self.model= model

        # Configuration par défaut
        self.config = {
            "layer_name": "decoder.6.weight",  # Couche cible
            "watermark_size": 8,       # Nombre de bits du watermark
            "spreading_factor": 4,     # Facteur d'étalement (S)
            "variance": 1.0, #0.025277,         # Variance pour la distribution Laplacienne
            "lr": 1e-4,
            "epochs": 30,
            "beta_kl":1.0
        }

        self.saved_keys = {}

    def _get_target_layer(self, model):
        """Récupère le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Paramètre {self.config['layer_name']} introuvable.")

    @staticmethod
    def _get_rand_bits(size):
        """Génère une séquence binaire aléatoire."""
        return [random.randint(0, 1) for _ in range(size)]

    def keygen(self, watermark_size):
        """
        Génère le watermark et les signes correspondants.
        Returns:
            wat_signs: Watermark en {-1, +1}
            watermark: Watermark binaire en {0, 1}
        """
        watermark = torch.tensor(self._get_rand_bits(watermark_size)).float().to(self.device)
        wat_signs = torch.tensor([-1.0 if i == 0 else 1.0 for i in watermark], dtype=torch.float).to(self.device)
        return wat_signs, watermark

    def generate_laplacian_pseudo_sequence(self, s, l, loc=0.0, scale=1.0, seed=None):
        """
        Génère une séquence pseudo-aléatoire avec distribution Laplacienne.
        
        Args:
            s: Spreading factor
            l: Taille du watermark
            loc: Paramètre de localisation (moyenne)
            scale: Paramètre d'échelle (diversité)
            seed: Graine aléatoire pour la reproductibilité
        
        Returns:
            torch.Tensor: Séquence pseudo de taille (s * l,)
        """
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)

        total_size = s * l
        laplace_dist = Laplace(loc=torch.tensor(loc), scale=torch.tensor(scale))
        pseudo_seq = laplace_dist.sample((total_size,)).to(self.device)
        
        return pseudo_seq

    def generate_weight_sequence(self, watermark_signs, pseudo_seq, S):
        """
        Génère la séquence de poids avec le facteur d'étalement S.
        wm_j = u_i · s_j
        """
        l = len(watermark_signs)
        n = len(pseudo_seq)

        if n < l * S:
            raise ValueError(f"Spreading sequence length ({n}) must be >= l*S ({l * S})")

        weight_seq = torch.zeros(l * S, dtype=torch.float32).to(self.device)

        for i in range(1, l + 1):
            start_idx = (i - 1) * S
            end_idx = i * S

            for j in range(start_idx, end_idx):
                weight_seq[j] = watermark_signs[i - 1] * pseudo_seq[j]

        return weight_seq

    def replace_weights(self, model, weight_sequence, selected_indices=None):
        """
        Remplace les poids sélectionnés avec les valeurs du watermark.
        """
        n = len(weight_sequence)
        target_layer = self._get_target_layer(model)
        
        weight_tensor = target_layer.data
        original_shape = weight_tensor.shape
        flat_weights = weight_tensor.flatten()
        total_weights = flat_weights.numel()

        if n > total_weights:
            raise ValueError(f"Sequence length {n} exceeds total weights {total_weights}")

        if selected_indices is None:
            selected_indices = random.sample(range(total_weights), n)

        for i, idx in enumerate(selected_indices):
            flat_weights[idx] = weight_sequence[i]

        target_layer.data = flat_weights.reshape(original_shape)
        return selected_indices

    def pick_weights_by_indices(self, model, selected_indices):
        """
        Extrait les poids sélectionnés d'une couche.
        """
        target_layer = self._get_target_layer(model)
        flat_weights = target_layer.data.flatten()
        selected_weights = flat_weights[selected_indices]
        return selected_weights

    def calculate_watermark_bits(self, selected_weights, spreading_sequence, S):
        """
        Calcule les bits du watermark à partir des poids extraits.
        """
        watermark_length = len(spreading_sequence) // S

        if len(spreading_sequence) != watermark_length * S:
            raise ValueError(f"Spreading sequence length must be divisible by S")

        watermark_bits = torch.zeros(watermark_length, dtype=torch.float32).to(self.device)
        correlation_sums = torch.zeros(watermark_length, dtype=torch.float32).to(self.device)

        for i in range(1, watermark_length + 1):
            start_idx = (i - 1) * S
            end_idx = i * S

            s_slice = spreading_sequence[start_idx:end_idx]
            w_slice = selected_weights[start_idx:end_idx]

            correlation_sum = torch.sum(s_slice * w_slice)
            correlation_sums[i - 1] = correlation_sum

            bit_value = 1.0 if correlation_sum >= 0 else 0.0
            watermark_bits[i - 1] = bit_value

        return watermark_bits, correlation_sums

    @staticmethod
    def _compute_ber(extracted, target):
        """Calcule le Bit Error Rate."""
        return (extracted != target).float().mean().item()

    def embed(self, dataloader):
        """
        Incorpore le watermark Laplacien pendant le finetuning.
        """
        print(f"--- Démarrage Embedding LAPLACIAN ({self.config['layer_name']}) ---")
        print(f"Watermark size: {self.config['watermark_size']} bits")
        print(f"Spreading factor: {self.config['spreading_factor']}")

        watermarked_model = self.model
        watermarked_model.train()

        # 1. Génération des clés
        watermark_signs, watermark = self.keygen(self.config["watermark_size"])
        
        # Calcul de gamma (scale parameter)
        gamma = self.config["variance"] / np.sqrt(2)
        
        # Génération de la séquence pseudo-aléatoire Laplacienne
        pseudo_seq = self.generate_laplacian_pseudo_sequence(
            s=self.config['spreading_factor'],
            l=self.config['watermark_size'],
            loc=0.0,
            scale=gamma
        )
        
        # Génération de la séquence de poids
        weight_seq = self.generate_weight_sequence(
            watermark_signs=watermark_signs,
            pseudo_seq=pseudo_seq,
            S=self.config['spreading_factor']
        )
        
        print(f"Weight sequence length: {len(weight_seq)}")

        # 2. Remplacement initial des poids
        selected_indices = self.replace_weights(watermarked_model, weight_seq)
        print(f"Selected {len(selected_indices)} weight positions")

        # 3. Optimiseur
        optimizer = torch.optim.AdamW(watermarked_model.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()

        # 4. Boucle d'entraînement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            
            for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)

                optimizer.zero_grad()

                recon, mu, logvar = watermarked_model(clean_images)

                # Reconstruction loss
                l_recon = F.mse_loss(recon, clean_images)

                # KL divergence
                l_kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

                # VAE total task loss
                l_main = l_recon + self.config["beta_kl"] * l_kl


                l_main.backward()
                optimizer.step()

                # C. Re-application du watermark (freeze weights)
                self.replace_weights(
                    model=watermarked_model,
                    weight_sequence=weight_seq,
                    selected_indices=selected_indices
                )

                # D. Calcul du BER
                extracted_weights = self.pick_weights_by_indices(watermarked_model, selected_indices)
                extracted_wm, _ = self.calculate_watermark_bits(
                    extracted_weights, pseudo_seq, self.config['spreading_factor']
                )
                ber = self._compute_ber(extracted_wm, watermark)

                pbar.set_postfix(L_Main=f"{l_main.item():.4f}", BER=f"{ber:.4f}")

        # Sauvegarde des clés
        self.saved_keys = {
            "watermark": watermark,
            "watermark_signs": watermark_signs,
            "pseudo_seq": pseudo_seq,
            "weight_seq": weight_seq,
            "selected_indices": selected_indices,
            "watermarked_model": watermarked_model
        }
        
        print(f"\n✅ Embedding terminé! BER final: {ber:.4f}")
        torch.save(self.saved_keys, "laplacian_VAE_model_checkpoint.pt")
        return watermarked_model

    def extract(self, model=None):
        """
        Extrait le watermark d'un modèle suspect.
        """
        if model is None:
            model = self.saved_keys["watermarked_model"]

        watermark = self.saved_keys["watermark"]
        pseudo_seq = self.saved_keys["pseudo_seq"]
        selected_indices = self.saved_keys["selected_indices"]

        # 1. Extraction des poids
        try:
            extracted_weights = self.pick_weights_by_indices(model, selected_indices)
        except ValueError:
            print("⚠️ Couche cible introuvable dans le modèle suspect.")
            return 1.0, None

        # 2. Calcul du watermark extrait
        extracted_wm, correlation_sums = self.calculate_watermark_bits(
            extracted_weights, pseudo_seq, self.config['spreading_factor']
        )

        # 3. Calcul du BER
        ber = self._compute_ber(extracted_wm, watermark)

        print(f"BER Extrait: {ber:.4f}")
        return ber, extracted_wm


/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
from torchvision import transforms

import os
import torch
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load from Hugging Face (no Google Drive issues)
print("Loading dataset...")


from datasets import load_from_disk
hf_dataset = load_from_disk("celeba_local")
# os.makedirs("./celeba_images/all", exist_ok=True)
# for i, item in enumerate(hf_dataset):
#     item['image'].save(f"./celeba_images/all/{i:06d}.jpg")
# del hf_dataset


class CelebAWrapper(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        if self.transform:
            image = self.transform(image)
        return image, 0

dataset = CelebAWrapper(hf_dataset, transform)
# del hf_dataset
# gc.collect()
# dataset = datasets.ImageFolder("./celeba_images", transform=transform)
print("Dataset loaded!")

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print("loader loaded!")


Loading dataset...
Dataset loaded!
loader loaded!


In [4]:
# --- EXEMPLE D'EXÉCUTION ---

latent_dim = 200

# Initialize the model
model = VAE(latent_dim=latent_dim)

# Load the trained weights
model_path = "./vae_celeba_latent_200_epochs_10_batch_64_subset_80000.pth"
model.load_state_dict(torch.load(model_path))
model.to(device)


# 2. Embedding Laplacian
laplacian_defense = LaplacianVAE(model, device=device)

watermarked_model = laplacian_defense.embed(dataloader)

--- Démarrage Embedding LAPLACIAN (decoder.6.weight) ---
Watermark size: 8 bits
Spreading factor: 4
Weight sequence length: 32
Selected 32 weight positions


Epoch 30/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:27<00:00, 36.07it/s, BER=0.0000, L_Main=0.2957]



✅ Embedding terminé! BER final: 0.0000


In [5]:
# 3. Test d'extraction sur le modèle watermarké
print("\n--- Test Extraction sur modèle watermarké ---")
ber, extracted_wm = laplacian_defense.extract()



--- Test Extraction sur modèle watermarké ---
BER Extrait: 0.0000


In [6]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack_laplacian(laplacian_obj, dataloader, epochs=5, lr=1e-4):
    """
    Tente de transférer la fonctionnalité du modèle Laplacian vers un modèle vierge.
    Vérifie si la marque (basée sur les poids) survit.
    """
    device = laplacian_obj.device
    checkpoint = torch.load("laplacian_VAE_model_checkpoint.pt", weights_only=False)

    # 1. Teacher (Gelé)
    teacher = checkpoint["watermarked_model"]
    teacher.eval()
    for p in teacher.parameters():
        p.requires_grad = False

    # 2. Student (Vierge - Même architecture)
    print("\n--- Initialisation du Student ---")
    student=VAE(latent_dim=latent_dim)
    student.load_state_dict(torch.load(model_path))
    student.to(device)
    student.train()

    # Sanity Checks
    teacher_ber, _ = laplacian_obj.extract(teacher)
    student_ber, _ = laplacian_obj.extract(student)
    print(f"[Check] BER Teacher: {teacher_ber:.4f}")
    print(f"[Check] BER Student (Avant): {student_ber:.4f}")

    optimizer = AdamW(student.parameters(), lr=lr)

    history = {"loss": [], "ber": []}

    print(f"\n--- Distillation Laplacian ({epochs} epochs) ---")
    a=0

    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)


            # B. Distillation (Output Matching)
            with torch.no_grad():
                target_pred,_,_ = teacher(clean_images)

            student_pred,_,_ = student(clean_images)

            loss1 = F.mse_loss(student_pred, target_pred)
            loss2=F.mse_loss(student_pred, clean_images)

            loss = 0.9* loss1 +0.1* loss2

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss=loss.item())

        # C. Vérification du BER après chaque epoch
        current_ber, pred_wm = laplacian_obj.extract(student)
        history["ber"].append(current_ber)
        history["loss"].append(running_loss / len(dataloader))

        print(f"👉 Fin Epoch {epoch+1} | Loss: {loss.item():.4f} | BER Student: {current_ber:.4f} err_wat: {nn.BCELoss()(pred_wm, checkpoint["watermark"]).item()}")
        if current_ber==0.0 and a>=1:
            print("✅ Marque récupérée avec succès par distillation !")
            break
        elif current_ber==0.0 and a<1 :
            a+=1
        else:
            a=0

    return student, history

In [ ]:
# 4. Attaque par Distillation
student_model, attack_stats = run_distillation_attack_laplacian(
    laplacian_defense, 
    dataloader, 
    epochs=100
)


--- Initialisation du Student ---
BER Extrait: 0.0000
BER Extrait: 0.3750
[Check] BER Teacher: 0.0000
[Check] BER Student (Avant): 0.3750

--- Distillation Laplacian (100 epochs) ---


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:25<00:00, 37.10it/s, Loss=0.0364]


BER Extrait: 0.3750
👉 Fin Epoch 1 | Loss: 0.0364 | BER Student: 0.3750 err_wat: 37.5


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:26<00:00, 36.67it/s, Loss=0.0357]


BER Extrait: 0.3750
👉 Fin Epoch 2 | Loss: 0.0357 | BER Student: 0.3750 err_wat: 37.5


...

Epoch 12: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:26<00:00, 36.55it/s, Loss=0.0313]


BER Extrait: 0.5000
👉 Fin Epoch 12 | Loss: 0.0313 | BER Student: 0.5000 err_wat: 50.0


Epoch 13: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3166/3166 [01:25<00:00, 36.91it/s, Loss=0.0361]


BER Extrait: 0.5000
👉 Fin Epoch 13 | Loss: 0.0361 | BER Student: 0.5000 err_wat: 50.0


Epoch 14:  62%|████████████████████████████████████████████████████████████████████████████████████████████████▎                                                          | 1967/3166 [00:55<00:33, 36.18it/s, Loss=0.0327]

In [ ]:
# 5. Visualisation des résultats
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# BER over epochs
axes[0].plot(attack_stats["ber"], marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('BER')
axes[0].set_title('BER Evolution During Distillation Attack')
axes[0].axhline(y=0.5, color='r', linestyle='--', label='Random guess (0.5)')
axes[0].legend()
axes[0].grid(True)

# Loss over epochs
axes[1].plot(attack_stats["loss"], marker='s', color='orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Over Epochs')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 6. Comparaison finale
print("\n" + "="*50)
print("RÉSUMÉ FINAL")
print("="*50)

teacher_ber, _ = laplacian_defense.extract(laplacian_defense.saved_keys["watermarked_unet"])
student_ber, _ = laplacian_defense.extract(student_model)

print(f"\nBER Teacher (watermarké): {teacher_ber:.4f}")
print(f"BER Student (après distillation): {student_ber:.4f}")



In [ ]:
%%sql


In [ ]:
%%sql
